In [1]:
import pymssql
import pandas as pd
import json
import os
import pickle
import numpy as np
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.layers.core import Dense
from keras.utils import np_utils
from keras import backend as K
from keras.models import load_model

%matplotlib inline
import matplotlib.pyplot as plt
from keras import backend as K


from keras.layers.core import Dense, Activation, Dropout, Flatten
# from keras.layers.convolutional import Convolution2D
# from keras.layers.convolutional import MaxPooling2D
from keras.callbacks import ModelCheckpoint

from keras.optimizers import Adam
from sklearn.metrics import log_loss, accuracy_score, f1_score
from sklearn.metrics import confusion_matrix, precision_score,recall_score


Using TensorFlow backend.


In [2]:
server = "41.221.51.164"
user = "sa"
password = "X$OUDDVC8)"
database = "eOXEGEN"
prev_database = "eOXEGEN_P"
pd.set_option("display.max_columns", 150)

In [3]:
con = pymssql.connect(server, user, password, database)
cursor = con.cursor()
con_prev = pymssql.connect(server, user, password, prev_database)
cursor_prev= con_prev.cursor()

In [4]:
query="""
SELECT COUNT(*) AS count,sum(CLAIMED_AMOUNT) AS total_sum  FROM eOXEGEN.dbo.INS_RC_CLAIM_INWARD WHERE MEMBERSHIP_NO='AARKEWATER2017110002570AA'
"""

new_query="""
SELECT SUM(CLAIMED_AMOUNT * EXCHANGE_RATE) AS TOTAL_CLAIM_AMOUNT, SUM(SETTLED_AMOUNT * EXCHANGE_RATE) AS ADMISSIBLE_AMOUNT,
SUM((CLAIMED_AMOUNT * EXCHANGE_RATE)-(SETTLED_AMOUNT * EXCHANGE_RATE)) AS INADMISSIBLE_AMOUNT FROM eOXEGEN.dbo.INS_RC_CLAIM_INWARD
        WHERE CLAIM_IS_ACTIVE=1 AND CLAIMED_AMOUNT>0 AND INS_COMP_ID='2017070311422763433871010'
        AND (CASE WHEN 
                CLAIM_CATEGORY='RC' AND CLAIM_TYPE='OP' THEN ADMISSION_DATE 
                ELSE
                (CASE WHEN CLAIM_CATEGORY='RC' AND CLAIM_TYPE='IP' THEN DISCHARGE_DATE
                ELSE
                ENTRY_TIME
                END)  
            END) >= '2017-04-04' AND
            (CASE WHEN 
                CLAIM_CATEGORY='RC' AND CLAIM_TYPE='OP' THEN ADMISSION_DATE 
                ELSE
                (CASE WHEN CLAIM_CATEGORY='RC' AND CLAIM_TYPE='IP' THEN DISCHARGE_DATE
                ELSE
                ENTRY_TIME
                END)  
            END)
             <='2018-02-02'
"""
cursor.execute(new_query)
result2, result3, result4 = cursor.fetchall()[0]

In [6]:
def function1(res1,res2,res3):
    return res1,res2,res3

In [9]:
abc,c,d=function1(result2, result3, result4)

In [10]:
print(abc)

395539738.25000000000


In [5]:
print(result2,result3, result4)
# if str()
print(type(result2), type(float(result3)))

395539738.25000000000 395342738.25000000000 197000.00000000000
<class 'decimal.Decimal'> <class 'float'>


In [4]:
# THESE ARE THE LIST OF QUERY VAULTS:

query_individual = """SELECT MAX(MEMBERSHIP_NO) AS MEMBERSHIP_NO,tab1.POLICY_CODE, MAX(tab1.CORPORATE_ID) AS CORPORATE_ID,
tab1.EMPLOYEE_CODE, DATEDIFF(MONTH,MAX(tab1.FIRST_ENROLLMENT_DATE),MAX(tab1.POLICY_VALID_UPTO)) AS STAY_PERIOD FROM (
SELECT MEMBERSHIP_NO, POLICY_CODE, MEMBER_ID, PRI_MEMBER_ID, MEMBER_IS_VIP, MEMBER_DEP_REL,PLAN_ID, SCHEME_ID, CORPORATE_ID,
EMPLOYEE_CODE, FIRST_ENROLLMENT_DATE, CONTINUITY_FROM, POLICY_VALID_FROM, POLICY_VALID_UPTO, AGENCY_CODE, PROPOSER_ID,
UPDATED_DATE, ENDR_TYPE,ENDR_BY, ENDR_DATE, ENDR_NO, MEMBERSHIP_STATUS
FROM eOXEGEN.dbo.INS_MEMBER_POLICY
WHERE INS_COMPANY_ID ='2017070311422763433871010' AND MEMBERSHIP_IS_ACTIVE=1 AND
POLICY_VALID_UPTO <= DATEADD(month, -1, GETDATE()) AND CORPORATE_ID IS NULL
) tab1 WHERE tab1.MEMBER_DEP_REL='A' GROUP BY tab1.EMPLOYEE_CODE, tab1.POLICY_CODE"""

query_corporate = """SELECT tab1.MEMBERSHIP_NO, tab1.POLICY_CODE, MAX(tab1.CORPORATE_ID) AS CORPORATE_ID,
MAX(tab1.EMPLOYEE_CODE) AS EMPLOYEE_CODE,
DATEDIFF(MONTH,MAX(tab1.FIRST_ENROLLMENT_DATE),MAX(tab1.POLICY_VALID_UPTO)) AS STAY_PERIOD FROM (
SELECT MEMBERSHIP_NO, POLICY_CODE, MEMBER_ID, PRI_MEMBER_ID, MEMBER_IS_VIP, MEMBER_DEP_REL,PLAN_ID, SCHEME_ID, CORPORATE_ID,
EMPLOYEE_CODE, FIRST_ENROLLMENT_DATE, CONTINUITY_FROM, POLICY_VALID_FROM, POLICY_VALID_UPTO, AGENCY_CODE, PROPOSER_ID,
UPDATED_DATE, ENDR_TYPE,ENDR_BY, ENDR_DATE, ENDR_NO, MEMBERSHIP_STATUS
FROM eOXEGEN.dbo.INS_MEMBER_POLICY
WHERE INS_COMPANY_ID ='2017070311422763433871010' AND MEMBERSHIP_IS_ACTIVE=1 AND
POLICY_VALID_UPTO <= DATEADD(month, -1, GETDATE()) AND CORPORATE_ID IS NOT NULL
) tab1 WHERE MEMBER_DEP_REL='A' GROUP BY tab1.MEMBERSHIP_NO, tab1.POLICY_CODE"""


individual_joined="""
SELECT individual.*,grouped.STAY_PERIOD FROM
(SELECT MAX(MEMBERSHIP_NO) AS MEMBERSHIP_NO,tab1.POLICY_CODE, MAX(tab1.CORPORATE_ID) AS CORPORATE_ID,
tab1.EMPLOYEE_CODE, DATEDIFF(MONTH,MAX(tab1.FIRST_ENROLLMENT_DATE),MAX(tab1.POLICY_VALID_UPTO)) AS STAY_PERIOD FROM (
SELECT MEMBERSHIP_NO, POLICY_CODE, MEMBER_ID, PRI_MEMBER_ID, MEMBER_IS_VIP, MEMBER_DEP_REL,PLAN_ID, SCHEME_ID, CORPORATE_ID,
EMPLOYEE_CODE, FIRST_ENROLLMENT_DATE, CONTINUITY_FROM, POLICY_VALID_FROM, POLICY_VALID_UPTO, AGENCY_CODE, PROPOSER_ID,
UPDATED_DATE, ENDR_TYPE,ENDR_BY, ENDR_DATE, ENDR_NO, MEMBERSHIP_STATUS
FROM eOXEGEN.dbo.INS_MEMBER_POLICY
WHERE INS_COMPANY_ID ='2017070311422763433871010' AND MEMBERSHIP_IS_ACTIVE=1 AND
POLICY_VALID_UPTO <= DATEADD(month, -1, GETDATE()) AND CORPORATE_ID IS NULL
) tab1 WHERE tab1.MEMBER_DEP_REL='A' GROUP BY tab1.EMPLOYEE_CODE, tab1.POLICY_CODE) grouped
INNER JOIN
eOXEGEN.dbo.INS_MEMBER_POLICY individual ON individual.POLICY_CODE LIKE
CONCAT('%',CAST(SUBSTRING(grouped.POLICY_CODE,1,20) AS VARCHAR),'%') AND
grouped.EMPLOYEE_CODE=individual.EMPLOYEE_CODE
"""


corporate_joined="""
SELECT corporate.*,grouped.STAY_PERIOD FROM
(SELECT tab1.MEMBERSHIP_NO, tab1.POLICY_CODE, MAX(tab1.CORPORATE_ID) AS CORPORATE_ID,
MAX(tab1.EMPLOYEE_CODE) AS EMPLOYEE_CODE,
DATEDIFF(MONTH,MAX(tab1.FIRST_ENROLLMENT_DATE),MAX(tab1.POLICY_VALID_UPTO)) AS STAY_PERIOD FROM (
SELECT MEMBERSHIP_NO, POLICY_CODE, MEMBER_ID, PRI_MEMBER_ID, MEMBER_IS_VIP, MEMBER_DEP_REL,PLAN_ID, SCHEME_ID, CORPORATE_ID,
EMPLOYEE_CODE, FIRST_ENROLLMENT_DATE, CONTINUITY_FROM, POLICY_VALID_FROM, POLICY_VALID_UPTO, AGENCY_CODE, PROPOSER_ID,
UPDATED_DATE, ENDR_TYPE,ENDR_BY, ENDR_DATE, ENDR_NO, MEMBERSHIP_STATUS
FROM eOXEGEN.dbo.INS_MEMBER_POLICY
WHERE INS_COMPANY_ID ='2017070311422763433871010' AND MEMBERSHIP_IS_ACTIVE=1 AND
POLICY_VALID_UPTO <= DATEADD(month, -1, GETDATE()) AND CORPORATE_ID IS NOT NULL
) tab1 WHERE MEMBER_DEP_REL='A' GROUP BY tab1.MEMBERSHIP_NO, tab1.POLICY_CODE) grouped
INNER JOIN
eOXEGEN.dbo.INS_MEMBER_POLICY corporate ON corporate.POLICY_CODE LIKE
CONCAT('%',CAST(SUBSTRING(grouped.POLICY_CODE,1,20) AS VARCHAR),'%') AND
grouped.MEMBERSHIP_NO=corporate.MEMBERSHIP_NO
"""

# We joined the INS_MEMBER_POLICY AND INS_MEMBER_POLICY table here, INS_PREMIUM_INVOICE table is to be added
combined_individual="""
SELECT table1.*, table2.MEMBER_LNAME, table2.MEMBER_PREFIX, table2.MEMBER_DISPLAY_NAME, table2.MEMBER_PRI_ADD_LINE1,
table2.MEMBER_PRI_ADD_LINE2, table2.MEMBER_PRI_CITY_ID, table2.MEMBER_PRI_STATE_ID, table2.MEMBER_AGE, table2.MEMBER_BIRTH_PLACE,
table2.MEMBER_ADMINISTRATIVE_SEX, table2.MEMBER_MARITAL_STATUS, table2.MEMBER_WEIGHT, table2.MEMBER_HEIGHT, table2.MEMBER_BLOOD_GROUP,
table2.MEMBER_RH_FACTOR FROM
(SELECT individual.MEMBERSHIP_NO, individual.POLICY_CODE, individual.MEMBER_ID, individual.PRI_MEMBER_ID,
individual.MEMBER_IS_VIP, individual.MEMBER_DEP_REL,individual.PLAN_ID, individual.SCHEME_ID,
individual.CORPORATE_ID,individual.EMPLOYEE_CODE, individual.FIRST_ENROLLMENT_DATE, individual.CONTINUITY_FROM,
individual.POLICY_VALID_FROM,individual.POLICY_VALID_UPTO, individual.AGENCY_CODE, individual.PROPOSER_ID,
individual.UPDATED_DATE, individual.ENDR_TYPE,individual.ENDR_BY, individual.ENDR_DATE,
individual.ENDR_NO, individual.MEMBERSHIP_STATUS,
grouped.STAY_PERIOD FROM
(SELECT MAX(tab1.MEMBERSHIP_NO) AS MEMBERSHIP_NO,tab1.POLICY_CODE, MAX(tab1.CORPORATE_ID) AS CORPORATE_ID,
tab1.EMPLOYEE_CODE, DATEDIFF(MONTH,MAX(tab1.FIRST_ENROLLMENT_DATE),MAX(tab1.POLICY_VALID_UPTO)) AS STAY_PERIOD
FROM (
SELECT * FROM eOXEGEN.dbo.INS_MEMBER_POLICY
WHERE INS_COMPANY_ID ='2017070311422763433871010' AND MEMBERSHIP_IS_ACTIVE=1 AND
POLICY_VALID_UPTO <= DATEADD(month, -1, GETDATE()) AND CORPORATE_ID IS NULL
) tab1 WHERE tab1.MEMBER_DEP_REL='A' GROUP BY tab1.EMPLOYEE_CODE, tab1.POLICY_CODE) grouped
INNER JOIN
eOXEGEN.dbo.INS_MEMBER_POLICY individual ON individual.POLICY_CODE LIKE
CONCAT('%',CAST(SUBSTRING(grouped.POLICY_CODE,1,20) AS VARCHAR),'%') AND
grouped.EMPLOYEE_CODE=individual.EMPLOYEE_CODE) table1
INNER JOIN
eOXEGEN.dbo.INS_MEMBER table2 ON table1.MEMBER_ID=table2.MEMBER_ID
"""


combined_corporate="""
SELECT table1.*, table2.MEMBER_LNAME, table2.MEMBER_PREFIX, table2.MEMBER_DISPLAY_NAME, table2.MEMBER_PRI_ADD_LINE1,
table2.MEMBER_PRI_ADD_LINE2, table2.MEMBER_PRI_CITY_ID, table2.MEMBER_PRI_STATE_ID, table2.MEMBER_AGE,
table2.MEMBER_BIRTH_PLACE,table2.MEMBER_ADMINISTRATIVE_SEX,table2.MEMBER_MARITAL_STATUS,table2.MEMBER_WEIGHT,
table2.MEMBER_HEIGHT, table2.MEMBER_BLOOD_GROUP,table2.MEMBER_RH_FACTOR FROM
(SELECT corporate.MEMBERSHIP_NO, corporate.POLICY_CODE, corporate.MEMBER_ID, corporate.PRI_MEMBER_ID,
corporate.MEMBER_IS_VIP, corporate.MEMBER_DEP_REL,corporate.PLAN_ID, corporate.SCHEME_ID,
corporate.CORPORATE_ID,corporate.EMPLOYEE_CODE, corporate.FIRST_ENROLLMENT_DATE, corporate.CONTINUITY_FROM,
corporate.POLICY_VALID_FROM,corporate.POLICY_VALID_UPTO, corporate.AGENCY_CODE, corporate.PROPOSER_ID,
corporate.UPDATED_DATE, corporate.ENDR_TYPE,corporate.ENDR_BY, corporate.ENDR_DATE, corporate.ENDR_NO,
corporate.MEMBERSHIP_STATUS,
grouped.STAY_PERIOD FROM
(SELECT tab1.MEMBERSHIP_NO, tab1.POLICY_CODE, MAX(tab1.CORPORATE_ID) AS CORPORATE_ID,
MAX(tab1.EMPLOYEE_CODE) AS EMPLOYEE_CODE,
DATEDIFF(MONTH,MAX(tab1.FIRST_ENROLLMENT_DATE),MAX(tab1.POLICY_VALID_UPTO)) AS STAY_PERIOD FROM (
SELECT * FROM eOXEGEN.dbo.INS_MEMBER_POLICY
WHERE INS_COMPANY_ID ='2017070311422763433871010' AND MEMBERSHIP_IS_ACTIVE=1 AND
POLICY_VALID_UPTO <= DATEADD(month, -1, GETDATE()) AND CORPORATE_ID IS NOT NULL
) tab1 WHERE MEMBER_DEP_REL='A' GROUP BY tab1.MEMBERSHIP_NO, tab1.POLICY_CODE) grouped
INNER JOIN
eOXEGEN.dbo.INS_MEMBER_POLICY corporate ON corporate.POLICY_CODE LIKE
CONCAT('%',CAST(SUBSTRING(grouped.POLICY_CODE,1,20) AS VARCHAR),'%') AND
grouped.MEMBERSHIP_NO=corporate.MEMBERSHIP_NO) table1
INNER JOIN
eOXEGEN.dbo.INS_MEMBER table2 ON table1.MEMBER_ID=table2.MEMBER_ID
"""


prev_individual="""
SELECT table1.*, table2.MEMBER_LNAME, table2.MEMBER_PREFIX, table2.MEMBER_DISPLAY_NAME, table2.MEMBER_PRI_ADD_LINE1,
table2.MEMBER_PRI_ADD_LINE2, table2.MEMBER_PRI_CITY_ID, table2.MEMBER_PRI_STATE_ID, table2.MEMBER_AGE, table2.MEMBER_BIRTH_PLACE,
table2.MEMBER_ADMINISTRATIVE_SEX, table2.MEMBER_MARITAL_STATUS, table2.MEMBER_WEIGHT, table2.MEMBER_HEIGHT, table2.MEMBER_BLOOD_GROUP,
table2.MEMBER_RH_FACTOR FROM
(SELECT individual.MEMBERSHIP_NO, individual.POLICY_CODE, individual.MEMBER_ID, individual.PRI_MEMBER_ID,
individual.MEMBER_IS_VIP, individual.MEMBER_DEP_REL,individual.PLAN_ID, individual.SCHEME_ID,
individual.CORPORATE_ID,individual.EMPLOYEE_CODE, individual.FIRST_ENROLLMENT_DATE, individual.CONTINUITY_FROM, individual.POLICY_VALID_FROM,
individual.POLICY_VALID_UPTO, individual.AGENCY_CODE, individual.PROPOSER_ID, individual.UPDATED_DATE, individual.ENDR_TYPE,
individual.ENDR_BY, individual.ENDR_DATE, individual.ENDR_NO, individual.MEMBERSHIP_STATUS,
grouped.STAY_PERIOD FROM
(SELECT MAX(MEMBERSHIP_NO) AS MEMBERSHIP_NO,tab1.POLICY_CODE, MAX(tab1.CORPORATE_ID) AS CORPORATE_ID,
tab1.EMPLOYEE_CODE, DATEDIFF(MONTH,MAX(tab1.FIRST_ENROLLMENT_DATE),MAX(tab1.POLICY_VALID_UPTO)) AS STAY_PERIOD FROM (
SELECT * FROM eOXEGEN.dbo.INS_MEMBER_POLICY_PREV
WHERE INS_COMPANY_ID ='2017070311422763433871010' AND MEMBERSHIP_IS_ACTIVE=1 AND
POLICY_VALID_UPTO <= DATEADD(month, -1, GETDATE()) AND CORPORATE_ID IS NULL
) tab1 WHERE tab1.MEMBER_DEP_REL='A' GROUP BY tab1.EMPLOYEE_CODE, tab1.POLICY_CODE) grouped
INNER JOIN
eOXEGEN.dbo.INS_MEMBER_POLICY_PREV individual ON individual.POLICY_CODE LIKE
CONCAT('%',CAST(SUBSTRING(grouped.POLICY_CODE,1,20) AS VARCHAR),'%') AND
grouped.EMPLOYEE_CODE=individual.EMPLOYEE_CODE) table1
INNER JOIN
eOXEGEN.dbo.INS_MEMBER table2 ON table1.MEMBER_ID=table2.MEMBER_ID
"""

prev_corporate="""
SELECT table1.*, table2.MEMBER_LNAME, table2.MEMBER_PREFIX, table2.MEMBER_DISPLAY_NAME, table2.MEMBER_PRI_ADD_LINE1,
table2.MEMBER_PRI_ADD_LINE2, table2.MEMBER_PRI_CITY_ID, table2.MEMBER_PRI_STATE_ID, table2.MEMBER_AGE, table2.MEMBER_BIRTH_PLACE,
table2.MEMBER_ADMINISTRATIVE_SEX, table2.MEMBER_MARITAL_STATUS, table2.MEMBER_WEIGHT, table2.MEMBER_HEIGHT, table2.MEMBER_BLOOD_GROUP,
table2.MEMBER_RH_FACTOR FROM
(SELECT corporate.MEMBERSHIP_NO, corporate.POLICY_CODE, corporate.MEMBER_ID, corporate.PRI_MEMBER_ID,
corporate.MEMBER_IS_VIP, corporate.MEMBER_DEP_REL,corporate.PLAN_ID, corporate.SCHEME_ID,
corporate.CORPORATE_ID,corporate.EMPLOYEE_CODE, corporate.FIRST_ENROLLMENT_DATE, corporate.CONTINUITY_FROM,
corporate.POLICY_VALID_FROM,corporate.POLICY_VALID_UPTO, corporate.AGENCY_CODE, corporate.PROPOSER_ID,
corporate.UPDATED_DATE, corporate.ENDR_TYPE,corporate.ENDR_BY, corporate.ENDR_DATE, corporate.ENDR_NO,
corporate.MEMBERSHIP_STATUS,
grouped.STAY_PERIOD FROM
(SELECT tab1.MEMBERSHIP_NO, tab1.POLICY_CODE, MAX(tab1.CORPORATE_ID) AS CORPORATE_ID,
MAX(tab1.EMPLOYEE_CODE) AS EMPLOYEE_CODE,
DATEDIFF(MONTH,MAX(tab1.FIRST_ENROLLMENT_DATE),MAX(tab1.POLICY_VALID_UPTO)) AS STAY_PERIOD FROM (
SELECT *
FROM eOXEGEN.dbo.INS_MEMBER_POLICY
WHERE INS_COMPANY_ID ='2017070311422763433871010' AND MEMBERSHIP_IS_ACTIVE=1 AND
POLICY_VALID_UPTO <= DATEADD(month, -1, GETDATE()) AND CORPORATE_ID IS NOT NULL
) tab1 WHERE MEMBER_DEP_REL='A' GROUP BY tab1.MEMBERSHIP_NO, tab1.POLICY_CODE) grouped
INNER JOIN
eOXEGEN.dbo.INS_MEMBER_POLICY corporate ON corporate.POLICY_CODE LIKE
CONCAT('%',CAST(SUBSTRING(grouped.POLICY_CODE,1,20) AS VARCHAR),'%') AND
grouped.MEMBERSHIP_NO=corporate.MEMBERSHIP_NO) table1
INNER JOIN
eOXEGEN.dbo.INS_MEMBER table2 ON table1.MEMBER_ID=table2.MEMBER_ID
"""

In [5]:
# THESE ARE THE ACTUAL QUERIES:

combined_individual="""
SELECT table1.*, table2.MEMBER_LNAME, table2.MEMBER_PREFIX, table2.MEMBER_DISPLAY_NAME, table2.MEMBER_PRI_ADD_LINE1,
table2.MEMBER_PRI_ADD_LINE2, table2.MEMBER_PRI_CITY_ID, table2.MEMBER_PRI_STATE_ID, table2.MEMBER_AGE, table2.MEMBER_BIRTH_PLACE,
table2.MEMBER_ADMINISTRATIVE_SEX, table2.MEMBER_MARITAL_STATUS, table2.MEMBER_WEIGHT, table2.MEMBER_HEIGHT, table2.MEMBER_BLOOD_GROUP,
table2.MEMBER_RH_FACTOR FROM
    (SELECT individual.MEMBERSHIP_NO,individual.POLICY_CODE, individual.MEMBER_ID, individual.PRI_MEMBER_ID,
    individual.MEMBER_IS_VIP, individual.MEMBER_DEP_REL,individual.PLAN_ID, individual.SCHEME_ID,
    individual.CORPORATE_ID,individual.EMPLOYEE_CODE, individual.FIRST_ENROLLMENT_DATE, individual.CONTINUITY_FROM,
    individual.POLICY_VALID_FROM,individual.POLICY_VALID_UPTO, individual.AGENCY_CODE, individual.PROPOSER_ID,
    individual.UPDATED_DATE, individual.ENDR_TYPE,individual.ENDR_BY, individual.ENDR_DATE,
    individual.ENDR_NO, individual.MEMBERSHIP_STATUS,grouped.STAY_PERIOD FROM
        (SELECT MAX(tab1.MEMBERSHIP_NO) AS MEMBERSHIP_NO,tab1.POLICY_CODE, MAX(tab1.CORPORATE_ID) AS CORPORATE_ID,
        tab1.EMPLOYEE_CODE, DATEDIFF(MONTH,MAX(tab1.FIRST_ENROLLMENT_DATE),MAX(tab1.POLICY_VALID_UPTO)) AS STAY_PERIOD
        FROM (
            SELECT * FROM eOXEGEN.dbo.INS_MEMBER_POLICY
            WHERE INS_COMPANY_ID ='2017070311422763433871010' AND MEMBERSHIP_IS_ACTIVE=1 AND
            POLICY_VALID_UPTO <= DATEADD(month, -6, GETDATE()) AND CORPORATE_ID IS NULL
            ) tab1
        WHERE tab1.MEMBER_DEP_REL='A' GROUP BY tab1.EMPLOYEE_CODE, tab1.POLICY_CODE) grouped
    INNER JOIN
    eOXEGEN.dbo.INS_MEMBER_POLICY individual ON individual.POLICY_CODE LIKE
    CONCAT('%',CAST(SUBSTRING(grouped.POLICY_CODE,1,20) AS VARCHAR),'%') AND
    grouped.EMPLOYEE_CODE=individual.EMPLOYEE_CODE) table1
INNER JOIN
eOXEGEN.dbo.INS_MEMBER table2 ON table1.MEMBER_ID=table2.MEMBER_ID
"""

combined_corporate="""
SELECT table1.*, table2.MEMBER_LNAME, table2.MEMBER_PREFIX, table2.MEMBER_DISPLAY_NAME, table2.MEMBER_PRI_ADD_LINE1,
table2.MEMBER_PRI_ADD_LINE2, table2.MEMBER_PRI_CITY_ID, table2.MEMBER_PRI_STATE_ID, table2.MEMBER_AGE,
table2.MEMBER_BIRTH_PLACE,table2.MEMBER_ADMINISTRATIVE_SEX,table2.MEMBER_MARITAL_STATUS,table2.MEMBER_WEIGHT,
table2.MEMBER_HEIGHT, table2.MEMBER_BLOOD_GROUP,table2.MEMBER_RH_FACTOR FROM
(SELECT corporate.MEMBERSHIP_NO, corporate.POLICY_CODE, corporate.MEMBER_ID, corporate.PRI_MEMBER_ID,
corporate.MEMBER_IS_VIP, corporate.MEMBER_DEP_REL,corporate.PLAN_ID, corporate.SCHEME_ID,
corporate.CORPORATE_ID,corporate.EMPLOYEE_CODE, corporate.FIRST_ENROLLMENT_DATE, corporate.CONTINUITY_FROM,
corporate.POLICY_VALID_FROM,corporate.POLICY_VALID_UPTO, corporate.AGENCY_CODE, corporate.PROPOSER_ID,
corporate.UPDATED_DATE, corporate.ENDR_TYPE,corporate.ENDR_BY, corporate.ENDR_DATE, corporate.ENDR_NO,
corporate.MEMBERSHIP_STATUS,
grouped.STAY_PERIOD FROM
(SELECT MAX(tab1.MEMBERSHIP_NO) AS MEMBERSHIP_NO, tab1.POLICY_CODE, tab1.CORPORATE_ID,
MAX(tab1.EMPLOYEE_CODE) AS EMPLOYEE_CODE,
DATEDIFF(MONTH,MAX(tab1.FIRST_ENROLLMENT_DATE),MAX(tab1.POLICY_VALID_UPTO)) AS STAY_PERIOD FROM (
SELECT * FROM eOXEGEN.dbo.INS_MEMBER_POLICY
WHERE INS_COMPANY_ID ='2017070311422763433871010' AND MEMBERSHIP_IS_ACTIVE=1 AND
POLICY_VALID_UPTO <= DATEADD(month, -6, GETDATE()) AND CORPORATE_ID IS NOT NULL
) tab1 WHERE MEMBER_DEP_REL='A' GROUP BY tab1.CORPORATE_ID, tab1.POLICY_CODE) grouped
INNER JOIN
eOXEGEN.dbo.INS_MEMBER_POLICY corporate ON corporate.POLICY_CODE LIKE
CONCAT('%',CAST(SUBSTRING(grouped.POLICY_CODE,1,20) AS VARCHAR),'%') AND
grouped.CORPORATE_ID=corporate.CORPORATE_ID) table1
INNER JOIN
eOXEGEN.dbo.INS_MEMBER table2 ON table1.MEMBER_ID=table2.MEMBER_ID
"""

prev_individual="""
SELECT table1.*, table2.MEMBER_LNAME, table2.MEMBER_PREFIX, table2.MEMBER_DISPLAY_NAME, table2.MEMBER_PRI_ADD_LINE1,
table2.MEMBER_PRI_ADD_LINE2, table2.MEMBER_PRI_CITY_ID, table2.MEMBER_PRI_STATE_ID, table2.MEMBER_AGE, table2.MEMBER_BIRTH_PLACE,
table2.MEMBER_ADMINISTRATIVE_SEX, table2.MEMBER_MARITAL_STATUS, table2.MEMBER_WEIGHT, table2.MEMBER_HEIGHT, table2.MEMBER_BLOOD_GROUP,
table2.MEMBER_RH_FACTOR FROM
(SELECT individual.MEMBERSHIP_NO, individual.POLICY_CODE, individual.MEMBER_ID, individual.PRI_MEMBER_ID,
individual.MEMBER_IS_VIP, individual.MEMBER_DEP_REL,individual.PLAN_ID, individual.SCHEME_ID,
individual.CORPORATE_ID,individual.EMPLOYEE_CODE, individual.FIRST_ENROLLMENT_DATE, individual.CONTINUITY_FROM, individual.POLICY_VALID_FROM,
individual.POLICY_VALID_UPTO, individual.AGENCY_CODE, individual.PROPOSER_ID, individual.UPDATED_DATE, individual.ENDR_TYPE,
individual.ENDR_BY, individual.ENDR_DATE, individual.ENDR_NO, individual.MEMBERSHIP_STATUS,
grouped.STAY_PERIOD FROM
(SELECT MAX(MEMBERSHIP_NO) AS MEMBERSHIP_NO,tab1.POLICY_CODE, MAX(tab1.CORPORATE_ID) AS CORPORATE_ID,
tab1.EMPLOYEE_CODE, DATEDIFF(MONTH,MAX(tab1.FIRST_ENROLLMENT_DATE),MAX(tab1.POLICY_VALID_UPTO)) AS STAY_PERIOD FROM (
SELECT * FROM eOXEGEN.dbo.INS_MEMBER_POLICY_PREV
WHERE INS_COMPANY_ID ='2017070311422763433871010' AND MEMBERSHIP_IS_ACTIVE=1 AND
POLICY_VALID_UPTO <= DATEADD(month, -6, GETDATE()) AND CORPORATE_ID IS NULL
) tab1 WHERE tab1.MEMBER_DEP_REL='A' GROUP BY tab1.EMPLOYEE_CODE, tab1.POLICY_CODE) grouped
INNER JOIN
eOXEGEN.dbo.INS_MEMBER_POLICY_PREV individual ON individual.POLICY_CODE LIKE
CONCAT('%',CAST(SUBSTRING(grouped.POLICY_CODE,1,20) AS VARCHAR),'%') AND
grouped.EMPLOYEE_CODE=individual.EMPLOYEE_CODE) table1
INNER JOIN
eOXEGEN.dbo.INS_MEMBER table2 ON table1.MEMBER_ID=table2.MEMBER_ID
"""

prev_corporate="""
SELECT table1.*, table2.MEMBER_LNAME, table2.MEMBER_PREFIX, table2.MEMBER_DISPLAY_NAME, table2.MEMBER_PRI_ADD_LINE1,
table2.MEMBER_PRI_ADD_LINE2, table2.MEMBER_PRI_CITY_ID, table2.MEMBER_PRI_STATE_ID, table2.MEMBER_AGE, table2.MEMBER_BIRTH_PLACE,
table2.MEMBER_ADMINISTRATIVE_SEX, table2.MEMBER_MARITAL_STATUS, table2.MEMBER_WEIGHT, table2.MEMBER_HEIGHT, table2.MEMBER_BLOOD_GROUP,
table2.MEMBER_RH_FACTOR FROM
(SELECT corporate.MEMBERSHIP_NO, corporate.POLICY_CODE, corporate.MEMBER_ID, corporate.PRI_MEMBER_ID,
corporate.MEMBER_IS_VIP, corporate.MEMBER_DEP_REL,corporate.PLAN_ID, corporate.SCHEME_ID,
corporate.CORPORATE_ID,corporate.EMPLOYEE_CODE, corporate.FIRST_ENROLLMENT_DATE, corporate.CONTINUITY_FROM,
corporate.POLICY_VALID_FROM,corporate.POLICY_VALID_UPTO, corporate.AGENCY_CODE, corporate.PROPOSER_ID,
corporate.UPDATED_DATE, corporate.ENDR_TYPE,corporate.ENDR_BY, corporate.ENDR_DATE, corporate.ENDR_NO,
corporate.MEMBERSHIP_STATUS,
grouped.STAY_PERIOD FROM
(SELECT tab1.MEMBERSHIP_NO, tab1.POLICY_CODE, MAX(tab1.CORPORATE_ID) AS CORPORATE_ID,
MAX(tab1.EMPLOYEE_CODE) AS EMPLOYEE_CODE,
DATEDIFF(MONTH,MAX(tab1.FIRST_ENROLLMENT_DATE),MAX(tab1.POLICY_VALID_UPTO)) AS STAY_PERIOD FROM (
SELECT *
FROM eOXEGEN.dbo.INS_MEMBER_POLICY
WHERE INS_COMPANY_ID ='2017070311422763433871010' AND MEMBERSHIP_IS_ACTIVE=1 AND
POLICY_VALID_UPTO <= DATEADD(month, -6, GETDATE()) AND CORPORATE_ID IS NOT NULL
) tab1 WHERE MEMBER_DEP_REL='A' GROUP BY tab1.MEMBERSHIP_NO, tab1.POLICY_CODE) grouped
INNER JOIN
eOXEGEN.dbo.INS_MEMBER_POLICY corporate ON corporate.POLICY_CODE LIKE
CONCAT('%',CAST(SUBSTRING(grouped.POLICY_CODE,1,20) AS VARCHAR),'%') AND
grouped.MEMBERSHIP_NO=corporate.MEMBERSHIP_NO) table1
INNER JOIN
eOXEGEN.dbo.INS_MEMBER table2 ON table1.MEMBER_ID=table2.MEMBER_ID
"""

premium_invoice_query="""
SELECT TOP 1000 * FROM eOXEGEN.dbo.INS_MEMBER_POLICY P INNER JOIN
(SELECT INVOICE_ID, REF_NO, INVOICE_NO, INVOICE_DATE, PROPOSER_ID, PREMIUM, MODE_OF_PAYMENT, TOTAL_MEMBERS
,PERIOD_FROM,PERIOD_TO, STATUS, PREM_FREQ_TYPE, PRINT_AMOUNT, ENTRY_TIME, IS_ACTIVE, COST_CENTER_CODE, PENALTY_AMT
FROM eOXEGEN.dbo.INS_PREMIUM_INVOICE) I
ON
P.POLICY_VALID_FROM=I.PERIOD_FROM
and P.PROPOSER_ID=I.PROPOSER_ID and P.POLICY_VALID_UPTO=I.PERIOD_TO
and P.MEMBERSHIP_IS_ACTIVE=1
and I.STATUS<>'X'
"""

In [6]:
df1 = pd.read_sql(combined_individual, con)
df2 = pd.read_sql(combined_corporate, con)
df3= pd.read_sql(prev_individual, con)
df4= pd.read_sql(prev_corporate, con)
con.close()
# df11 = pd.read_sql(individual_prev_joined, con_prev)
# df2 = pd.read_sql(corporate_prev_joined, con_prev)
# con_prev.close()

In [7]:
concating=[df1,df2,df3,df4]
df=pd.concat(concating)

In [8]:
file_name="image1.pkl"
data_path = os.path.join('/home/rasello/projects/AAR_JUPYTER',file_name)

In [9]:
with open(data_path, 'wb') as file:
    pickle.dump(df, file, protocol=pickle.HIGHEST_PROTOCOL)

In [10]:
# import json
# data_path_Datadict = os.path.join('static/json_pics/past_actual_records.json',file_name)

# with open('%s' % data_path_Datadict, 'w') as outfile:
#     json.dump(past_actual_records, outfile)
with open(data_path, 'rb') as handle:
    df = pickle.load(handle)

In [11]:
df.shape

(52859, 38)

In [12]:
df.head(3)

,MEMBERSHIP_NO,POLICY_CODE,MEMBER_ID,PRI_MEMBER_ID,MEMBER_IS_VIP,MEMBER_DEP_REL,PLAN_ID,SCHEME_ID,CORPORATE_ID,EMPLOYEE_CODE,FIRST_ENROLLMENT_DATE,CONTINUITY_FROM,POLICY_VALID_FROM,POLICY_VALID_UPTO,AGENCY_CODE,PROPOSER_ID,UPDATED_DATE,ENDR_TYPE,ENDR_BY,ENDR_DATE,ENDR_NO,MEMBERSHIP_STATUS,STAY_PERIOD,MEMBER_LNAME,MEMBER_PREFIX,MEMBER_DISPLAY_NAME,MEMBER_PRI_ADD_LINE1,MEMBER_PRI_ADD_LINE2,MEMBER_PRI_CITY_ID,MEMBER_PRI_STATE_ID,MEMBER_AGE,MEMBER_BIRTH_PLACE,MEMBER_ADMINISTRATIVE_SEX,MEMBER_MARITAL_STATUS,MEMBER_WEIGHT,MEMBER_HEIGHT,MEMBER_BLOOD_GROUP,MEMBER_RH_FACTOR
0,AARTZINDVI2018030000007AA,TZIN/18/PA174/000008/01,2018030810545732677251150,None,0.0,A,2018021612280874728131030,2018021612310646591511500,None,3.0,2017-01-08,2017-01-08,2018-01-08,2019-01-07,2017101912445440920241060,2018030718440654264721060,None,None,None,NaT,None,US,24,HILL,MR,MR EDWARD HILL,TANGA,TANGA,TZ.TAN.TANG138,TZ.TAN,32.0,None,M,None,75.0,188.0,None,None
1,AARTZINDVI2018030000007CA,TZIN/18/PA174/000008/01,2018030810545762237311800,2018030810545732677251150,0.0,C,2018021612280874728131030,2018021612310646591511500,None,3.0,2017-01-08,2017-01-08,2018-01-08,2019-01-07,2017101912445440920241060,2018030718440654264721060,None,None,None,NaT,None,US,24,HILL,MISS,MISS ARI HILL,TANGA,TANGA,TZ.TAN.TANG138,TZ.TAN,2.0,None,F,None,10.0,110.0,None,None
2,AARTZINDVI2018030000007CB,TZIN/18/PA174/000008/01,2018030810545818461412100,2018030810545732677251150,0.0,C,2018021612280874728131030,2018021612310646591511500,None,3.0,2018-01-08,2018-01-08,2018-01-08,2019-01-07,2017101912445440920241060,2018030718440654264721060,None,None,None,NaT,None,US,24,HILL,MISS,MISS GLORIA GRACE IMARA HILL,TANGA,TANGA,TZ.TAN.TANG138,TZ.TAN,3.0,None,F,None,15.0,120.0,None,None


In [ ]:
# def classify(x):
#     return int(round(x/12))

In [ ]:
# df['STAY_YEAR']=df['STAY_PERIOD'].apply(classify)

In [8]:
df.replace(r'\s+',np.nan, regex=True)
df.dropna(axis=0,how='all',inplace=True)

for i in df.columns:
    if((df[i].dtypes!= np.number) & (df[i].dtypes!= 'datetime64[ns]')):
        df.loc[df[i].isnull(),str(i)]='101010'
    if((df[i].dtypes==np.number) & (df[i].dtypes!= 'datetime64[ns]')):
        df.loc[df[i].isnull(),str(i)]=101010
    if(df[i].dtypes== 'datetime64[ns]'):
        df.loc[df[i].isnull(),str(i)]='1800-01-01'
        df[i] = pd.to_datetime(df[i])

In [9]:
df.head(3)

,MEMBERSHIP_NO,POLICY_CODE,MEMBER_ID,PRI_MEMBER_ID,MEMBER_IS_VIP,MEMBER_DEP_REL,PLAN_ID,SCHEME_ID,CORPORATE_ID,EMPLOYEE_CODE,FIRST_ENROLLMENT_DATE,CONTINUITY_FROM,POLICY_VALID_FROM,POLICY_VALID_UPTO,AGENCY_CODE,PROPOSER_ID,UPDATED_DATE,ENDR_TYPE,ENDR_BY,ENDR_DATE,ENDR_NO,MEMBERSHIP_STATUS,STAY_PERIOD,MEMBER_LNAME,MEMBER_PREFIX,MEMBER_DISPLAY_NAME,MEMBER_PRI_ADD_LINE1,MEMBER_PRI_ADD_LINE2,MEMBER_PRI_CITY_ID,MEMBER_PRI_STATE_ID,MEMBER_AGE,MEMBER_BIRTH_PLACE,MEMBER_ADMINISTRATIVE_SEX,MEMBER_MARITAL_STATUS,MEMBER_WEIGHT,MEMBER_HEIGHT,MEMBER_BLOOD_GROUP,MEMBER_RH_FACTOR
0,AARTZINDVI2018030000007AA,TZIN/18/PA174/000008/01,2018030810545732677251150,101010,0.0,A,2018021612280874728131030,2018021612310646591511500,101010,3.0,2017-01-08,2017-01-08,2018-01-08,2019-01-07,2017101912445440920241060,2018030718440654264721060,101010,101010,101010,1800-01-01,101010,US,24,HILL,MR,MR EDWARD HILL,TANGA,TANGA,TZ.TAN.TANG138,TZ.TAN,32.0,101010,M,101010,75.0,188.0,101010,101010
1,AARTZINDVI2018030000007CA,TZIN/18/PA174/000008/01,2018030810545762237311800,2018030810545732677251150,0.0,C,2018021612280874728131030,2018021612310646591511500,101010,3.0,2017-01-08,2017-01-08,2018-01-08,2019-01-07,2017101912445440920241060,2018030718440654264721060,101010,101010,101010,1800-01-01,101010,US,24,HILL,MISS,MISS ARI HILL,TANGA,TANGA,TZ.TAN.TANG138,TZ.TAN,2.0,101010,F,101010,10.0,110.0,101010,101010
2,AARTZINDVI2018030000007CB,TZIN/18/PA174/000008/01,2018030810545818461412100,2018030810545732677251150,0.0,C,2018021612280874728131030,2018021612310646591511500,101010,3.0,2018-01-08,2018-01-08,2018-01-08,2019-01-07,2017101912445440920241060,2018030718440654264721060,101010,101010,101010,1800-01-01,101010,US,24,HILL,MISS,MISS GLORIA GRACE IMARA HILL,TANGA,TANGA,TZ.TAN.TANG138,TZ.TAN,3.0,101010,F,101010,15.0,120.0,101010,101010


In [10]:
target = df['STAY_PERIOD']
features = df.drop(['STAY_PERIOD'], axis = 1)
feature_backup=features.copy()

In [11]:
for y in list(features.columns):
    if((features[y].dtypes!= np.number) & (features[y].dtypes!= 'datetime64[ns]')):
        features[y] = features[y].astype('category').cat.rename_categories(range(1, features[y].nunique()+1))
    if(features[y].dtypes == 'datetime64[ns]'):  
        features[y] = pd.to_datetime(features[y], format='%Y-%m-%d').dt.strftime(date_format='%Y%m%d')

In [12]:
for m in list(features.columns):
    features[m]=pd.to_numeric(features[m]) 

In [13]:

scale_features_std = StandardScaler()
features = features.values  # Returns a numpy array
target = target.values
features = scale_features_std.fit_transform(features)
features = pd.DataFrame(data=features,columns=list(feature_backup.columns))

features, target=shuffle(features, target)
X_train, X_test, y_train, y_test = train_test_split(features, target,
                                                   test_size=0.2, random_state=0)

In [14]:
features.head()

,MEMBERSHIP_NO,POLICY_CODE,MEMBER_ID,PRI_MEMBER_ID,MEMBER_IS_VIP,MEMBER_DEP_REL,PLAN_ID,SCHEME_ID,CORPORATE_ID,EMPLOYEE_CODE,FIRST_ENROLLMENT_DATE,CONTINUITY_FROM,POLICY_VALID_FROM,POLICY_VALID_UPTO,AGENCY_CODE,PROPOSER_ID,UPDATED_DATE,ENDR_TYPE,ENDR_BY,ENDR_DATE,ENDR_NO,MEMBERSHIP_STATUS,MEMBER_LNAME,MEMBER_PREFIX,MEMBER_DISPLAY_NAME,MEMBER_PRI_ADD_LINE1,MEMBER_PRI_ADD_LINE2,MEMBER_PRI_CITY_ID,MEMBER_PRI_STATE_ID,MEMBER_AGE,MEMBER_BIRTH_PLACE,MEMBER_ADMINISTRATIVE_SEX,MEMBER_MARITAL_STATUS,MEMBER_WEIGHT,MEMBER_HEIGHT,MEMBER_BLOOD_GROUP,MEMBER_RH_FACTOR
41756,-0.907890,1.058522,1.391063,1.839079,0.0,1.033422,1.248777,0.707917,1.232936,-0.554415,1.032213,0.898280,1.270434,0.109789,1.114210,1.133886,0.547975,0.456319,-0.583541,-0.609236,-0.488924,0.283892,-1.057615,-1.631120,-1.551316,-0.602520,-0.301556,-0.196425,-0.188858,-0.923649,-0.109575,0.924672,-0.133263,-0.671734,-0.726128,-0.134927,-0.135108
22465,0.368641,0.321109,-0.305659,0.364408,0.0,-0.067432,0.312182,-0.393030,0.271447,0.432221,0.238631,0.217742,-0.700373,-0.037080,0.771213,0.306414,0.547975,0.456319,-0.583541,-0.609236,-0.488924,0.283892,1.393633,0.621422,0.515986,-0.570378,-0.225654,-0.196425,-0.188858,1.013430,-0.109575,0.924672,-0.133263,-0.671734,-0.726128,-0.134927,-0.135108
10847,0.170020,-0.258287,-0.850432,-0.913981,0.0,-1.168286,-0.460509,-1.147966,-0.488800,1.140410,-2.240797,-4.926537,-0.763948,-0.125495,1.800203,-0.342585,0.547975,0.456319,-0.583541,-0.609236,-0.488924,0.283892,1.236683,0.621422,0.797155,1.658092,2.152639,-0.196425,-0.188858,1.450835,-0.109575,0.924672,-0.133263,1.798396,1.729290,-0.134927,-0.135108
17942,1.536261,0.926841,1.042371,-0.913981,0.0,-1.168286,1.061458,1.038201,1.054054,1.566805,-0.591669,0.239988,1.271281,-0.126670,0.634015,0.955412,-0.944087,-0.764831,2.224623,1.646498,3.258317,-3.136777,-1.314957,0.621422,0.861397,-0.570378,-0.225654,-0.196425,-0.188858,0.576025,-0.109575,0.924672,-0.133263,-0.671734,-0.726128,-0.134927,-0.135108
19204,-1.497124,0.808328,0.497340,1.072728,0.0,1.033422,0.850724,0.267538,0.897533,-0.867519,-0.591957,0.232377,-0.657990,0.021374,-0.875169,0.841837,0.547975,0.456319,-0.583541,-0.609236,-0.488924,0.283892,-0.470820,-0.129425,0.000451,-0.570378,-0.225654,-0.196425,-0.188858,-1.173595,-0.109575,-1.074926,-0.133263,-0.671734,-0.726128,-0.134927,-0.135108


In [15]:
def baseline_model():
    input_dim = X_train.shape[1]
    # num_classes = y_train.shape[1]
    print('We have {} entries in the training feature column'.format(input_dim))
    #Preparing a seuential model for a binary classification related problem
    model = Sequential()
    model.add(Dense(input_dim, kernel_initializer='normal', input_shape=(input_dim,), activation='relu'))
    model.add(Dense(90, kernel_initializer='normal', activation='relu'))
    model.add(Dense(210, kernel_initializer='normal', activation='relu'))
    model.add(Dense(120, kernel_initializer='normal',activation='relu'))
    # model.add(Dense(num_classes, activation='softmax'))
    model.add(Dense(1,kernel_initializer='normal'))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
#     model.summary()
    return model

In [16]:
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

seed = 7
np.random.seed(seed)
estimator = KerasRegressor(build_fn=baseline_model, epochs=10, batch_size=5, verbose=2)

#Defining the callback functions
callbacks = [EarlyStopping(monitor='val_loss', patience=8, mode ='auto')]  #, ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]

kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(estimator, X_test, y_test, cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

We have 37 entries in the training feature column
Epoch 1/10
 - 2s - loss: 90.2324 - acc: 0.0898
Epoch 2/10
 - 2s - loss: 71.9092 - acc: 0.1136
Epoch 3/10
 - 2s - loss: 58.2008 - acc: 0.1515
Epoch 4/10
 - 2s - loss: 54.0649 - acc: 0.1658
Epoch 5/10
 - 2s - loss: 53.8383 - acc: 0.1657
Epoch 6/10
 - 2s - loss: 49.3659 - acc: 0.1867
Epoch 7/10
 - 2s - loss: 44.7234 - acc: 0.1752
Epoch 8/10
 - 2s - loss: 38.0424 - acc: 0.2037
Epoch 9/10
 - 2s - loss: 42.5808 - acc: 0.1890
Epoch 10/10
 - 2s - loss: 34.6325 - acc: 0.2168
We have 37 entries in the training feature column
Epoch 1/10
 - 2s - loss: 83.3798 - acc: 0.0864
Epoch 2/10
 - 2s - loss: 65.7744 - acc: 0.1062
Epoch 3/10
 - 2s - loss: 57.9770 - acc: 0.1308
Epoch 4/10
 - 2s - loss: 51.8975 - acc: 0.1383
Epoch 5/10
 - 2s - loss: 47.0949 - acc: 0.1431
Epoch 6/10
 - 2s - loss: 45.5297 - acc: 0.1578
Epoch 7/10
 - 2s - loss: 41.4370 - acc: 0.1763
Epoch 8/10
 - 2s - loss: 39.5828 - acc: 0.2040
Epoch 9/10
 - 2s - loss: 38.2916 - acc: 0.2163
Epoch 

In [ ]:
print("Training...")
history = model.fit(X_train, y_train, epochs = 50, callbacks = callbacks, batch_size=16, validation_split=0.2, verbose=2)

In [17]:
plt.figure(figsize=(16,6))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model mean squared error loss')
plt.ylabel('mean squared error loss')
plt.xlabel('epoch')
plt.legend(['training set', 'validation set'], loc='upper right')
plt.savefig('train.png')
plt.show()

NameError: name 'history' is not defined

<Figure size 1152x432 with 0 Axes>

In [ ]:
model_name='aar_churn.h5'
model.save(os.path.join('/home/rasello/projects/AAR_JUPYTER',model_name))
K.clear_session()

In [ ]:
modelV = load_model(os.path.join('/home/rasello/projects/AAR_JUPYTER','aar_churn.h5'))

In [20]:
# Query to fetch all the in policy patients
active_individual="""
SELECT table1.*, table2.MEMBER_LNAME, table2.MEMBER_PREFIX, table2.MEMBER_DISPLAY_NAME, table2.MEMBER_PRI_ADD_LINE1,
table2.MEMBER_PRI_ADD_LINE2, table2.MEMBER_PRI_CITY_ID, table2.MEMBER_PRI_STATE_ID, table2.MEMBER_AGE, table2.MEMBER_BIRTH_PLACE,
table2.MEMBER_ADMINISTRATIVE_SEX, table2.MEMBER_MARITAL_STATUS, table2.MEMBER_WEIGHT, table2.MEMBER_HEIGHT, table2.MEMBER_BLOOD_GROUP,
table2.MEMBER_RH_FACTOR FROM (SELECT
MEMBERSHIP_NO, POLICY_CODE, MEMBER_ID, PRI_MEMBER_ID,MEMBER_IS_VIP, MEMBER_DEP_REL,PLAN_ID, SCHEME_ID,
CORPORATE_ID,EMPLOYEE_CODE, FIRST_ENROLLMENT_DATE, CONTINUITY_FROM,
POLICY_VALID_FROM,POLICY_VALID_UPTO, AGENCY_CODE, PROPOSER_ID,
UPDATED_DATE, ENDR_TYPE,ENDR_BY, ENDR_DATE,
ENDR_NO, MEMBERSHIP_STATUS FROM eOXEGEN.dbo.INS_MEMBER_POLICY
WHERE INS_COMPANY_ID ='2017070311422763433871010' AND MEMBERSHIP_IS_ACTIVE=1 AND
POLICY_VALID_UPTO > GETDATE() AND CORPORATE_ID IS NULL AND MEMBER_DEP_REL='A') table1
INNER JOIN
eOXEGEN.dbo.INS_MEMBER table2 ON table1.MEMBER_ID=table2.MEMBER_ID
"""

active_corporate="""
SELECT table1.*, table2.MEMBER_LNAME, table2.MEMBER_PREFIX, table2.MEMBER_DISPLAY_NAME, table2.MEMBER_PRI_ADD_LINE1,
table2.MEMBER_PRI_ADD_LINE2, table2.MEMBER_PRI_CITY_ID, table2.MEMBER_PRI_STATE_ID, table2.MEMBER_AGE,
table2.MEMBER_BIRTH_PLACE,table2.MEMBER_ADMINISTRATIVE_SEX,table2.MEMBER_MARITAL_STATUS,table2.MEMBER_WEIGHT,
table2.MEMBER_HEIGHT, table2.MEMBER_BLOOD_GROUP,table2.MEMBER_RH_FACTOR FROM

(SELECT MAX(tab1.MEMBERSHIP_NO) AS MEMBERSHIP_NO, tab1.POLICY_CODE, tab1.CORPORATE_ID,
MAX(tab1.EMPLOYEE_CODE) AS EMPLOYEE_CODE,MAX(tab1.MEMBER_ID) AS MEMBER_ID, MAX(tab1.PRI_MEMBER_ID) AS PRI_MEMBER_ID,
MAX(tab1.MEMBER_IS_VIP) AS MEMBER_IS_VIP, MAX(tab1.MEMBER_DEP_REL) AS MEMBER_DEP_REL,MAX(tab1.PLAN_ID) AS PLAN_ID,
MAX(tab1.SCHEME_ID) AS SCHEME_ID,MIN(tab1.FIRST_ENROLLMENT_DATE) AS FIRST_ENROLLMENT_DATE, MIN(tab1.CONTINUITY_FROM) AS CONTINUITY_FROM,
MIN(tab1.POLICY_VALID_FROM) AS POLICY_VALID_FROM, MAX(tab1.POLICY_VALID_UPTO) AS POLICY_VALID_UPTO, MAX(tab1.AGENCY_CODE) AS AGENCY_CODE,
MAX(tab1.PROPOSER_ID) AS PROPOSER_ID, MAX(tab1.UPDATED_DATE) AS UPDATED_DATE, MAX(tab1.ENDR_TYPE) AS ENDR_TYPE,
MAX(tab1.ENDR_BY) AS ENDR_BY, MAX(tab1.ENDR_DATE) AS ENDR_DATE, MAX(tab1.ENDR_NO) AS ENDR_NO, MAX(tab1.MEMBERSHIP_STATUS) AS MEMBERSHIP_STATUS
FROM (
    SELECT * FROM eOXEGEN.dbo.INS_MEMBER_POLICY
    WHERE INS_COMPANY_ID ='2017070311422763433871010' AND MEMBERSHIP_IS_ACTIVE=1 AND
    POLICY_VALID_UPTO > GETDATE() AND CORPORATE_ID IS NOT NULL) tab1
WHERE MEMBER_DEP_REL='A' GROUP BY tab1.CORPORATE_ID, tab1.POLICY_CODE) table1
INNER JOIN
eOXEGEN.dbo.INS_MEMBER table2 ON table1.MEMBER_ID=table2.MEMBER_ID
"""

prev_active_individual="""
SELECT table1.*, table2.MEMBER_LNAME, table2.MEMBER_PREFIX, table2.MEMBER_DISPLAY_NAME, table2.MEMBER_PRI_ADD_LINE1,
table2.MEMBER_PRI_ADD_LINE2, table2.MEMBER_PRI_CITY_ID, table2.MEMBER_PRI_STATE_ID, table2.MEMBER_AGE, table2.MEMBER_BIRTH_PLACE,
table2.MEMBER_ADMINISTRATIVE_SEX, table2.MEMBER_MARITAL_STATUS, table2.MEMBER_WEIGHT, table2.MEMBER_HEIGHT, table2.MEMBER_BLOOD_GROUP,
table2.MEMBER_RH_FACTOR FROM (SELECT
MEMBERSHIP_NO, POLICY_CODE, MEMBER_ID, PRI_MEMBER_ID,MEMBER_IS_VIP, MEMBER_DEP_REL,PLAN_ID, SCHEME_ID,
CORPORATE_ID,EMPLOYEE_CODE, FIRST_ENROLLMENT_DATE, CONTINUITY_FROM,
POLICY_VALID_FROM,POLICY_VALID_UPTO, AGENCY_CODE, PROPOSER_ID,
UPDATED_DATE, ENDR_TYPE,ENDR_BY, ENDR_DATE,
ENDR_NO, MEMBERSHIP_STATUS FROM eOXEGEN.dbo.INS_MEMBER_POLICY_PREV
WHERE INS_COMPANY_ID ='2017070311422763433871010' AND MEMBERSHIP_IS_ACTIVE=1 AND
POLICY_VALID_UPTO > GETDATE() AND CORPORATE_ID IS NULL AND MEMBER_DEP_REL='A') table1
INNER JOIN
eOXEGEN.dbo.INS_MEMBER table2 ON table1.MEMBER_ID=table2.MEMBER_ID
"""

prev_active_corporate="""
SELECT table1.*, table2.MEMBER_LNAME, table2.MEMBER_PREFIX, table2.MEMBER_DISPLAY_NAME, table2.MEMBER_PRI_ADD_LINE1,
table2.MEMBER_PRI_ADD_LINE2, table2.MEMBER_PRI_CITY_ID, table2.MEMBER_PRI_STATE_ID, table2.MEMBER_AGE,
table2.MEMBER_BIRTH_PLACE,table2.MEMBER_ADMINISTRATIVE_SEX,table2.MEMBER_MARITAL_STATUS,table2.MEMBER_WEIGHT,
table2.MEMBER_HEIGHT, table2.MEMBER_BLOOD_GROUP,table2.MEMBER_RH_FACTOR FROM

(SELECT MAX(tab1.MEMBERSHIP_NO) AS MEMBERSHIP_NO, tab1.POLICY_CODE, tab1.CORPORATE_ID,
MAX(tab1.EMPLOYEE_CODE) AS EMPLOYEE_CODE,MAX(tab1.MEMBER_ID) AS MEMBER_ID, MAX(tab1.PRI_MEMBER_ID) AS PRI_MEMBER_ID,
MAX(tab1.MEMBER_IS_VIP) AS MEMBER_IS_VIP, MAX(tab1.MEMBER_DEP_REL) AS MEMBER_DEP_REL,MAX(tab1.PLAN_ID) AS PLAN_ID,
MAX(tab1.SCHEME_ID) AS SCHEME_ID,MIN(tab1.FIRST_ENROLLMENT_DATE) AS FIRST_ENROLLMENT_DATE, MIN(tab1.CONTINUITY_FROM) AS CONTINUITY_FROM,
MIN(tab1.POLICY_VALID_FROM) AS POLICY_VALID_FROM, MAX(tab1.POLICY_VALID_UPTO) AS POLICY_VALID_UPTO, MAX(tab1.AGENCY_CODE) AS AGENCY_CODE,
MAX(tab1.PROPOSER_ID) AS PROPOSER_ID, MAX(tab1.UPDATED_DATE) AS UPDATED_DATE, MAX(tab1.ENDR_TYPE) AS ENDR_TYPE,
MAX(tab1.ENDR_BY) AS ENDR_BY, MAX(tab1.ENDR_DATE) AS ENDR_DATE, MAX(tab1.ENDR_NO) AS ENDR_NO, MAX(tab1.MEMBERSHIP_STATUS) AS MEMBERSHIP_STATUS
FROM (
    SELECT * FROM eOXEGEN.dbo.INS_MEMBER_POLICY_PREV
    WHERE INS_COMPANY_ID ='2017070311422763433871010' AND MEMBERSHIP_IS_ACTIVE=1 AND
    POLICY_VALID_UPTO > GETDATE() AND CORPORATE_ID IS NOT NULL) tab1
WHERE MEMBER_DEP_REL='A' GROUP BY tab1.CORPORATE_ID, tab1.POLICY_CODE) table1
INNER JOIN
eOXEGEN.dbo.INS_MEMBER table2 ON table1.MEMBER_ID=table2.MEMBER_ID
"""

In [21]:
con = pymssql.connect(server, user, password, database)
df1 = pd.read_sql(active_individual, con)
df2 = pd.read_sql(prev_active_individual, con)
df3= pd.read_sql(active_corporate, con)
df4= pd.read_sql(prev_active_corporate, con)
con.close()

In [22]:
part1 = [df1,df2]
part2 = [df3,df4]
individual_df=pd.concat(part1)
corporate_df=pd.concat(part2)
i_df=individual_df.copy()
c_df=corporate_df.copy()

In [23]:
individual_df.replace(r'\s+',np.nan, regex=True)
individual_df.dropna(axis=0,how='all',inplace=True)

for i in individual_df.columns:
    if((individual_df[i].dtypes!= np.number) & (individual_df[i].dtypes!= 'datetime64[ns]')):
        individual_df.loc[individual_df[i].isnull(),str(i)]='101010'
    if((individual_df[i].dtypes==np.number) & (individual_df[i].dtypes!= 'datetime64[ns]')):
        individual_df.loc[individual_df[i].isnull(),str(i)]=101010
    if(individual_df[i].dtypes== 'datetime64[ns]'):
        individual_df.loc[individual_df[i].isnull(),str(i)]='1800-01-01'
        individual_df[i] = pd.to_datetime(individual_df[i])
        

corporate_df.replace(r'\s+',np.nan, regex=True)
corporate_df.dropna(axis=0,how='all',inplace=True)

for i in corporate_df.columns:
    if((corporate_df[i].dtypes!= np.number) & (corporate_df[i].dtypes!= 'datetime64[ns]')):
        corporate_df.loc[corporate_df[i].isnull(),str(i)]='101010'
    if((corporate_df[i].dtypes==np.number) & (corporate_df[i].dtypes!= 'datetime64[ns]')):
        corporate_df.loc[corporate_df[i].isnull(),str(i)]=101010
    if(corporate_df[i].dtypes== 'datetime64[ns]'):
        corporate_df.loc[corporate_df[i].isnull(),str(i)]='1800-01-01'
        corporate_df[i] = pd.to_datetime(corporate_df[i])
        

In [24]:
for y in list(individual_df.columns):
    if((individual_df[y].dtypes!= np.number) & (individual_df[y].dtypes!= 'datetime64[ns]')):
        individual_df[y] = individual_df[y].astype('category').cat.rename_categories(range(1, individual_df[y].nunique()+1))
    if(individual_df[y].dtypes == 'datetime64[ns]'):  
        individual_df[y] = pd.to_datetime(individual_df[y], format='%Y-%m-%d').dt.strftime(date_format='%Y%m%d')

In [25]:
for y in list(corporate_df.columns):
    if((corporate_df[y].dtypes!= np.number) & (corporate_df[y].dtypes!= 'datetime64[ns]')):
        corporate_df[y] = corporate_df[y].astype('category').cat.rename_categories(range(1, corporate_df[y].nunique()+1))
    if(corporate_df[y].dtypes == 'datetime64[ns]'):  
        corporate_df[y] = pd.to_datetime(corporate_df[y], format='%Y-%m-%d').dt.strftime(date_format='%Y%m%d')

In [26]:
for m in list(individual_df.columns):
    individual_df[m]=pd.to_numeric(individual_df[m]) 

for n in list(corporate_df.columns):
    corporate_df[n]=pd.to_numeric(corporate_df[n]) 

In [27]:
individual_backup=individual_df.columns
corporate_backup=corporate_df.columns

In [28]:
scale_features_std = StandardScaler()
individual_df = individual_df.values  # Returns a numpy array
individual_df = scale_features_std.fit_transform(individual_df)
individual_df = pd.DataFrame(data=individual_df,columns=list(individual_backup))


corporate_df = corporate_df.values  # Returns a numpy array
corporate_df = scale_features_std.fit_transform(corporate_df)
corporate_df = pd.DataFrame(data=corporate_df,columns=list(corporate_backup))

In [29]:
# sample_input = X_test[0:100]
# sample_input.head()
# true_label = y_test[0:100]
# true_label

In [31]:
prediction = estimator.predict(individual_df)
# prediction_months = np.argmax(prediction, axis=1)

In [33]:
prediction_months=prediction
i_df['dum']= list(prediction_months)
# date_today=pd.Timestamp.today().date()
i_df['tdy']=pd.Timestamp.today().date()

i_df['ind']=pd.to_datetime(i_df['FIRST_ENROLLMENT_DATE'])
i_df['tdy']=pd.to_datetime(i_df['tdy'])

i_df['sm'] = (i_df.tdy - i_df.ind).map(lambda x: round(x.days/30))
i_df['rem'] = i_df.dum-i_df.sm

In [34]:
i_df.head()

,MEMBERSHIP_NO,POLICY_CODE,MEMBER_ID,PRI_MEMBER_ID,MEMBER_IS_VIP,MEMBER_DEP_REL,PLAN_ID,SCHEME_ID,CORPORATE_ID,EMPLOYEE_CODE,FIRST_ENROLLMENT_DATE,CONTINUITY_FROM,POLICY_VALID_FROM,POLICY_VALID_UPTO,AGENCY_CODE,PROPOSER_ID,UPDATED_DATE,ENDR_TYPE,ENDR_BY,ENDR_DATE,ENDR_NO,MEMBERSHIP_STATUS,MEMBER_LNAME,MEMBER_PREFIX,MEMBER_DISPLAY_NAME,MEMBER_PRI_ADD_LINE1,MEMBER_PRI_ADD_LINE2,MEMBER_PRI_CITY_ID,MEMBER_PRI_STATE_ID,MEMBER_AGE,MEMBER_BIRTH_PLACE,MEMBER_ADMINISTRATIVE_SEX,MEMBER_MARITAL_STATUS,MEMBER_WEIGHT,MEMBER_HEIGHT,MEMBER_BLOOD_GROUP,MEMBER_RH_FACTOR,dum,tdy,ind,sm,rem
0,AARTZINDVI2018050000203AA,TZIN/18/PA174/000184/02,2018053115483933481431270,None,0.0,A,2018021612280874728131030,2018021612310646591511500,None,255756348405,2018-05-30,2018-05-30,2019-05-30,2020-05-29,2017103016500546021501010,2018053114392484726191160,2019-05-30 10:51:26.330,None,None,NaT,None,US,OSORO,MR,MR GODWIN OTIENO OSORO,MBWENI,NA,TZ.DAR.DARE006,TZ.DAR,43.0,None,M,None,87.0,173.0,None,None,9.015937,2019-09-09,2018-05-30,16,-6.984063
1,AARTZINDVI2018030000008AA,TZIN/18/PA174/000009/02,2018030812023381746631010,None,0.0,A,2018021612280874728131030,2018021612312577854341220,None,RETAIL 05,2018-03-07,2018-03-07,2019-03-07,2020-03-06,2018030712120008610261010,2018030718444026999951660,2019-03-22 08:25:04.360,None,None,NaT,None,US,KIVUYO,MR,MR JOMBI KIVUYO,ARUSHA,ARUSHA,TZ.ARU.ARUS001,TZ.ARU,40.0,None,M,None,75.0,175.0,None,None,6.442649,2019-09-09,2018-03-07,18,-11.557351
2,AARTZINDVI2018030000011AA,TZIN/18/PA174/000012/02,2018031212302165370301070,None,0.0,A,2018021612280874728131030,2018030609405150954521650,None,TZ04266,2016-09-22,2016-09-22,2019-03-05,2020-03-04,2018030809540607979310500,2018030816072921652041120,2019-04-01 16:15:07.607,None,None,NaT,None,US,ABDALLAH,MRS,MRS ROSE ABDALLAH,DAR ES SALAAM,DAR ES SALAAM,TZ.DAR.DARE006,TZ.DAR,54.0,None,F,None,70.0,170.0,None,None,2.716462,2019-09-09,2016-09-22,36,-33.283538
3,AARTZINDVI2018030000012AA,TZIN/18/PA174/000010/02,2018031212435316850391160,None,0.0,A,2018021612280874728131030,2018021612310646591511500,None,TZ47578,2013-10-08,2013-10-08,2019-02-14,2020-02-13,2017103017063794835741030,2018030816082049483471160,2019-03-26 10:11:40.953,None,None,NaT,None,US,ALANDO,MR,MR CARILUS OKUMU ALANDO,DAR ES SALAAM,DAR ES SALAAM,TZ.DAR.DARE006,TZ.DAR,43.0,None,M,None,75.0,175.0,None,None,8.136183,2019-09-09,2013-10-08,72,-63.863817
4,AARTZINDVI2018050000166AA,TZIN/18/PA174/000152/02,2018052312094541215311530,None,0.0,A,2018021612280874728131030,2018030609411322462111290,None,TZ010724,2017-05-01,2019-05-10,2019-05-10,2020-05-09,2017103017211987864301010,2018052214485017421061590,2019-05-21 10:20:35.830,None,None,NaT,None,US,GUBARA,MRS,MRS OMSALAMA GUBARA,LIVINGSTONE/ MCHIKICHI,KARIAKOO,TZ.DAR.DARE006,TZ.DAR,53.0,None,F,None,85.0,169.0,None,None,3.164827,2019-09-09,2017-05-01,29,-25.835173


In [ ]:
pred = modelV.predict(corporate_df)
pred_months = np.argmax(pred, axis=1)
c_df['dum']= list(pred_months)
# date_today=pd.Timestamp.today().date()
c_df['tdy']=pd.Timestamp.today().date()

c_df['ind']=pd.to_datetime(c_df['FIRST_ENROLLMENT_DATE'])
c_df['tdy']=pd.to_datetime(c_df['tdy'])

c_df['sm'] = (c_df.tdy - c_df.ind).map(lambda x: round(x.days/30))
c_df['rem'] = c_df.dum-c_df.sm

In [ ]:
i_df.loc[i_df['rem'] < 0, 'rem'] = 0 #some unique value is set
i_df['num'] = 0
# new_list = []
new_list = i_df.groupby(['rem'])['num'].count().reset_index()
individual_remaining_months = (new_list.rem).tolist()
individual_users_number = (new_list.num).tolist()

In [ ]:
c_df.loc[c_df['rem'] < 0, 'rem'] = 0 #some unique value is set
c_df['num'] = 0
# new_list = []
corporate_list = c_df.groupby(['rem'])['num'].count().reset_index()
corporate_remaining_months = (corporate_list.rem).tolist()
corporate_users_number = (corporate_list.num).tolist()

In [ ]:
prediction = False
pred=False
K.clear_session()

In [ ]:
corporate_users_number